In [1]:
import arcade

GRAVITY = 0.8
SCREEN_WIDTH = 1280
SCREEN_HEIGHT = 720
SCREEN_TITLE = "Background Example"


CHARACTER_SCALING = 0.20
ENEMY_SCALING = 0.70
TILE_SCALING = 0.5
LAVA_SCALING = 0.7
SPIKE_SCALING = 0.4
PLAYER_MOVEMENT_SPEED = 5


class MenuView(arcade.View):
    def __init__(self):
        super().__init__()
        self.setup()

    def setup(self):
        self.background = arcade.load_texture("Space.png")

    def on_draw(self):
        self.clear()
        arcade.draw_texture_rect(
            self.background,
            arcade.LBWH(0, 0, SCREEN_WIDTH, SCREEN_HEIGHT),
        )

        arcade.draw_text(
            "Back 2 Mars",
            SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2,
            
            arcade.color.WHITE, font_size=50, anchor_x="center"
        )

        arcade.draw_text(
            "Click to continue",
            SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 - 75,
            arcade.color.WHITE, font_size=20, anchor_x="center"
        )

    def on_mouse_press(self, _x, _y, _button, _modifiers):
        instructions_view = InstructionView()
        self.window.show_view(instructions_view)


class InstructionView(arcade.View):
    def on_show_view(self):
        self.window.background_color = arcade.color.BLACK

    def on_draw(self):
        self.clear()
        arcade.draw_text(
            "MISSION: Help the Alien Escape the Facility!",
            SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2,
            arcade.color.WHITE, font_size=40, anchor_x="center"
        )
        arcade.draw_text(
            "Click to continue",
            SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 - 75,
            arcade.color.WHITE, font_size=20, anchor_x="center"
        )

    def on_mouse_press(self, _x, _y, _button, _modifiers):
        game_view = GameView()
        game_view.setup_level_1()
        self.window.show_view(game_view)


class GameOverView(arcade.View):
    def on_draw(self):
        self.clear()
        arcade.draw_text("You were caught!", SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2,
                         arcade.color.WHITE, 50, anchor_x="center")

        arcade.draw_text("Click to restart",
                         SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 - 75,
                         arcade.color.WHITE, 20, anchor_x="center")

    def on_mouse_press(self, x, y, button, modifiers):
        game = GameView()
        game.setup_level_1()
        self.window.show_view(game)

class GameView(arcade.View):
    def __init__(self):
        super().__init__()

        self.scene = None
        self.alien_sprite = None
        self.man_sprite = None

        self.alien_physics = None
        self.man_physics = None

        self.background = arcade.load_texture("background_lab.png")
        self.current_level = 1

   
    def setup_level_1(self):
        self.current_level = 1
        self.setup()

    def next_level(self):
        if self.current_level == 1:
            # FIXED: SHOW LEVEL 2 VIEW
            # HEREEEEE — LEVEL 2 FIX
            self.window.show_view(Level2View())
        else:
            self.window.show_view(GameOverView())

    def setup(self):
        self.scene = arcade.Scene()

        # Alien
        self.alien_sprite = arcade.Sprite("alien.png", CHARACTER_SCALING)
        self.alien_sprite.center_x = 64
        self.alien_sprite.center_y = 128
        self.scene.add_sprite("Alien", self.alien_sprite)

        # Man
        self.man_sprite = arcade.Sprite("man.png", ENEMY_SCALING)
        self.man_sprite.center_x = 1250
        self.man_sprite.center_y = 100
        self.scene.add_sprite("Man", self.man_sprite)

        # Floor
        for x in range(0, 1300, 60):
            wall = arcade.Sprite(":resources:/images/tiles/stoneCenter.png", TILE_SCALING)
            wall.center_x = x
            wall.center_y = 34
            self.scene.add_sprite("Walls", wall)

        # Floating slabs
        slab_positions = [
            [200, 150], [260, 150], [300, 150], [450, 200], [510,200], [665, 250], [700, 250], [745, 250], [800, 250]
        ]
        for pos in slab_positions:
            slab = arcade.Sprite(":resources:/images/tiles/stoneHalf_mid.png", TILE_SCALING)
            slab.position = pos
            self.scene.add_sprite("Slab", slab)

        
        for pos in [[260, 207], [515, 257], [750, 308]]:
            spike = arcade.Sprite(":resources:/images/tiles/spikes.png", SPIKE_SCALING)
            spike.position = pos
            self.scene.add_sprite("Hazard", spike)

        for pos in [[180, 35], [600, 35], [802, 35], [891, 35], [980, 35]]:
            lava = arcade.Sprite(":resources:/images/tiles/lavaTop_high.png", LAVA_SCALING)
            lava.position = pos
            self.scene.add_sprite("Hazard", lava)

        self.alien_physics = arcade.PhysicsEnginePlatformer(
            self.alien_sprite,
            [self.scene["Walls"], self.scene["Slab"]],
            gravity_constant=GRAVITY
        )

        self.man_physics = arcade.PhysicsEnginePlatformer(
            self.man_sprite,
            platforms=self.scene["Walls"],
            gravity_constant=GRAVITY
        )

    def on_draw(self):
        self.clear()
        arcade.draw_texture_rect(
            self.background,
            arcade.LBWH(0, 0, SCREEN_WIDTH, SCREEN_HEIGHT),
        )
        self.scene.draw()

    def on_key_press(self, key, modifiers):
        if key == arcade.key.LEFT:
            self.alien_sprite.change_x = -PLAYER_MOVEMENT_SPEED
        elif key == arcade.key.RIGHT:
            self.alien_sprite.change_x = PLAYER_MOVEMENT_SPEED
        elif key == arcade.key.UP and self.alien_physics.can_jump():
            self.alien_sprite.change_y = 15

    def on_key_release(self, key, modifiers):
        if key in (arcade.key.LEFT, arcade.key.RIGHT):
            self.alien_sprite.change_x = 0

    def on_update(self, delta_time):
        self.alien_physics.update()
        self.man_physics.update()

        # Simple chase AI
        if self.man_sprite.center_x < self.alien_sprite.center_x:
            self.man_sprite.change_x = 3.25
        else:
            self.man_sprite.change_x = -3.25

        # Hit by man
        if arcade.check_for_collision(self.man_sprite, self.alien_sprite):
            self.window.show_view(GameOverView())

        # Hit hazards
        hazard_list = self.scene["Hazard"]
        if arcade.check_for_collision_with_list(self.alien_sprite, hazard_list):
            self.window.show_view(GameOverView())

        # Reach right edge

        if self.alien_sprite.center_x < 10:
            self.alien_sprite.center_x = 10
       


def main():
    window = arcade.Window(SCREEN_WIDTH, SCREEN_HEIGHT, SCREEN_TITLE)
    menu = MenuView()
    window.show_view(menu)
    arcade.run()


if __name__ == "__main__":
    main()


C:\Users\mnsal\anaconda3\Lib\site-packages\arcade\exceptions.py:138: PerformanceWarning: draw_text is an extremely slow function for displaying text. Consider using Text objects instead.
  warnings.warn(message, warning_type)
